# Cystisoma corneas

## Inference

First open your heatmap in napari. 

In [ ]:
# import modules
import torchio as tio

# load in the nifti file with torchio
image_path = '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii'

image = tio.ScalarImage(image_path)



In [ ]:
image

In [ ]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [ ]:
import pandas as pd
# 2nd scan
#RR = pd.read_csv('/home/jake/projects/dhr/FEG20190213_01_corneas_resample_ratio.csv')
# 3rd scan
# RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_sp_FEG221129_366_corneas_estimating_resample_ratio.csv')
RR = pd.read_csv('resample_ratio_measurements/resample_ratio_measurement_Paraphronima_crassipes_f536_u1701837_head.csv')

RR

In [ ]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix


In [ ]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

In [ ]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance

In [ ]:
resample_ratio = mean_nearest_distance/10 #Makes average distance between features 10 voxels
resample_ratio

## Inference

In [ ]:
!python main.py infer -h #If you need help for parameters in infer

Make inference first with 1 direction in the x, y, and z axis before creating inference in 3 directions. This will produce three files: two csv files (one for the resampled space, and the other not), and a nifty file with predictions.

*Note: these files are currently saved to /home/jake/projects/dhr/output/

In [ ]:
#Inference with 1 direction in x, y, and z
# !python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 1 -ny 1 -nz 1

# Inference in 3 directions with 0.25 threshold
!python main.py infer configs/paraphronima_corneas.yaml -v '/media/jake/Dropbox_mirror/Smithsonian Dropbox/Jan Hemmi/hyperiid scans/Scans_nifti/Paraphronima_crassipes_f536_u1701837_head/Paraphronima_crassipes_f536_u1701837_head.nii' -m ./logs/paraphronima_corneas/lightning_logs/version_13/ -rr 3.1768202046528278 -nx 3 -ny 3 -nz 3 --average_threshold 0.25 -ipmv 0.25

You can then try in more directions (3 direction in x, y, z is usually best) and modify parameters such as max filter (-mxf) (although this usually doesn't work well with Cystisoma), average threshold (-at), and peak minimum values (-ipmv).

In [ ]:
#Attempts of inferences for 2nd scan (best model should be found on last commented line in this kernel)
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 1.5
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 2
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.01 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.03
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.04 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.05
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.1
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.3 
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.4
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 --infer_with_max_filter_fun
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.03 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.01 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.15 -at 0.01 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.01
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.02 
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.02
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.4 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.3 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.2 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.1 --infer_with_max_filter_fun
# !python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.05 --infer_with_max_filter_fun


#Best inference for 2nd scan
#!python main.py infer configs/cystisoma_corneas.yaml -v /media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii -m ./logs/cystisoma_corneas/lightning_logs/version_13/ -rr 2.588516765563356 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 

In [ ]:
#Scan 3 (not used)
# !python main.py infer configs/cystisoma_corneas.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_sp_FEG221129_366/FEG221129_366.nii' -m ./logs/cystisoma_corneas/lightning_logs/version_14/ -rr 2.3151474885249055 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5

You can then view your inference in napari. It might be useful to you, especially if you are trying many inferences with modified parameters, to evalute the performance of each and track estimated % of correct and incorrect inference points in an Excel spreadsheet for example.

In [ ]:
import napari
import numpy as np
import torchio as tio


image = tio.ScalarImage(image_path)

heatmap_path = './output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction.nii'
#Make sure to use resampled space csv here.
y_hat_path = 'output/Paraphronima_crassipes_f536_u1701837_head.logs_paraphronima_corneas_lightning_logs_version_13_checkpoints_last_x_3_y_3_z_3_average_threshold_0.25_prediction_peak_min_val_0_25_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
#viewer.add_image(image.numpy(), name='image')

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)


## Modifying inference

In software such as MatLab you can modify the inference made from your model by adding and deleting points. This will be useful in retraining your model later.

##  (Re)training 

In [ ]:
# We trained this model on a single scan, ran inference on it and then manually checked the labels. Now we are adding that labelled data to the dataset and retraining the model
# python check_data.py configs/cystisoma_corneas.yaml
# after updating the config to train for another 100,000 steps
# and changing the sigma from 2.5 to 2 (to suit the new data)
!python main.py train configs/cystisoma_corneas.yaml -w ./logs/cystisoma_corneas/lightning_logs/version_13/checkpoints/epoch=2809-step=699520.ckpt -ne 10

# Cystisoma rhabdoms

## Checking

Check hyperparameters for heatmap. Modify these need be.

In [ ]:
!python check_data.py ./configs/cystisoma_rhabdoms.yaml

In [ ]:
!python check_data.py -h

## (Re)training

After you finsihing checking the model you can train the model to rhabdoms and extend number of steps. 

In [ ]:
#If you want to see all arguments 
!python main.py train -h

You can try training multiple models with different starting weights if necessary.

In [ ]:
# !python main.py train configs/cystisoma_rhabdoms.yaml -w /home/jake/projects/dhr/logs/paraphronima_rhabdoms/lightning_logs/version_23/checkpoints/epoch=99-step=537600-v1.ckpt -ne 10
# !python main.py train configs/cystisoma_rhabdoms.yaml -w ./logs/paraphronima_rhabdoms_from_pretrained/lightning_logs/version_1/checkpoints/epoch=149-step=900000.ckpt #version_19
# !python main.py train ./configs/cystisoma_rhabdoms.yaml -w ./logs/fiddlercrab_rhabdoms_from_pretrained/lightning_logs/version_1/checkpoints/epoch=123-step=865024.ckpt #version_20 (have not run)
# !python main.py train ./configs/cystisoma_rhabdoms.yaml -w ./logs/cystisoma_rhabdoms/lightning_logs/version_13/checkpoints/epoch=62679-step=4971328.ckpt -ne 10 #version_21 (have not run)

## Inference

First open your heatmap in napari. Use the point selector to select some neighbouring rhabdoms and create a csv file of these selected points. You will use this to make your resampling ratio.

In [ ]:
# load in the nifti file with torchio
# 2nd scan
image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'

image = tio.ScalarImage(image_path)


In [ ]:
image

In [ ]:
# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')


In napari, label a few of the features (that are next to each other) to estimate the resample ratio, then save it as a csv file and load it in the next step


In [ ]:
import pandas as pd

#Read in your csv and assign it to the variable RR
# 2nd scan
RR = pd.read_csv('/home/jake/projects/dhr/Cystisoma_Hawaii_FEG20190213_rhabdoms_estimating_resampling_ratio.csv')

RR

In [ ]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-1', 'axis-2', 'axis-3']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix between points

In [ ]:
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR

In [ ]:
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance
resample_ratio = mean_nearest_distance/10
resample_ratio #Print your RR; use this to make your inference 

Now you can make your inference. Make sure you have the correct yaml, scan and model path, and have inserted your new resampling ratio. This will produce 3 files: 2 csv files (one resampled and the other not) with the predicted points, and a nifty file with your heatmap.

In [ ]:
# Me trying different inferences on version_17 
# Note:(-w for version_17 was cystisoma rhabdoms version_13 and starting sigma was only 2)
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.2 -at 0.2
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.3 -at 0.3
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.4 -at 0.4
!python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_17/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.2

In [ ]:
# Me trying different inferences for version_19 
# (Note:-w for version_19 was paraphronima_rhabdoms_from_pretrainied version_1 and starting sigma was changed to 4)
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.2 -at 0.2
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 1 -ny 1 -nz 1 -ipmv 0.3 -at 0.3
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 2 -ny 2 -nz 2 -ipmv 0.5 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.4
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.45 -at 0.4

# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.3 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.2 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.3
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.2
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.4 -at 0.1
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.15
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.25
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.35
# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.45

# !python main.py infer configs/cystisoma_rhabdoms.yaml -v '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii' -m ./logs/cystisoma_rhabdoms/lightning_logs/version_19/ -rr 2.152958176307709 -nx 3 -ny 3 -nz 3 -ipmv 0.45 -at 0.5





You can then view your inference in napari.

In [ ]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/2tb_ssd/mctv_data/Cystisoma_Hawaii_FEG20190213_01_head/FEG20190213_010.nii'
image = tio.ScalarImage(image_path)

# heatmap_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
heatmap_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_19_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'

#Make sure to use resampled space csv here.
# y_hat_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
y_hat_path = '/home/jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_19_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_45_method_center_of_mass.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
# viewer.add _image(image.numpy(), name='image') #If you want to view your scan also (not necessary)

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)

# Primno corneas

## Generate dataset

Convert your mat file into a csv file for corneas labels for each already labelled scan (save to raw_labels folder:'/home/jake/projects/dhr/dataset/raw_labels'). Create a csv called primno_corneas.csv (for structure of this csv see dhr: https://github.com/jakemanger/dhr)(save to data_source_specifiers folder: '/home/jake/projects/dhr/data_source_specifiers'). Insert the raw labels paths and image paths into this csv. 

In [ ]:
!python mctv_to_csv.py -h

# This method is not appropriate for Primno (currently) and does not include all 4 eyes, see steps below to convert mat to csv
# !python mctv_to_csv.py '/home/jake/projects/dhr/data_source_specifiers/primno_corneas.csv'

Use the next few blocks to explore your mat file. You should only need this once.

In [ ]:
# Use this block of code to examine dictionary keys in your mat file
import h5py

file_path = '/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat'
mat = h5py.File(file_path, 'r')

# List all top-level keys (variable names)
print("Keys (variable names) in the .mat file:")
keys = list(mat.keys())
print(keys)

# Use this block of code to view contents in save_dat
if isinstance(mat['save_dat'], h5py.Group):
    print("\n'save_dat' is a group. Listing its contents:")
    save_dat_group = mat['save_dat']
    group_keys = list(save_dat_group.keys())
    print(group_keys)

    for key in group_keys:
        item = save_dat_group[key]
        if isinstance(item, h5py.Dataset):
            # If it's a dataset, print its details
            print(f"\n'{key}' is a dataset.")
            print(f"Data type of {key}: {item.dtype}")
            print(f"Shape of {key}: {item.shape}")

            # Convert the dataset to a NumPy array if needed
            data_array = item[:]
            print(f"Data from {key}:\n", data_array)
        elif isinstance(item, h5py.Group):
            # If it's a group, print that it's a group
            print(f"\n'{key}' is a group.")
        else:
            print(f"'{key}' is of unknown type.")



In [ ]:
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as mat:
    save_dat_group = mat['save_dat']
    
    def explore_group(group):
        for name, item in group.items():
            if isinstance(item, h5py.Group):
                print(f"Group: {name}")
                explore_group(item) 
            elif isinstance(item, h5py.Dataset):
                print(f"Dataset: {name}, Shape: {item.shape}, Data Type: {item.dtype}")

    # Start exploring from save_dat_group
    explore_group(save_dat_group)



In [ ]:
# Use this block of code to explore the structure of groups (i.e. in this case you want to use this code to find the names of group to be used in the next block)
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    def print_structure(name, obj):
        print(f"{name}: {obj}")
    
    file.visititems(print_structure) 

In [ ]:
import pandas as pd

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    # Access the marked dataset using the correct path
    marked_data = file['save_dat/data/marked'][:, :1642] 

marked_df = pd.DataFrame(marked_data)

print(marked_df)


In [ ]:
# Use this block to investigate a specifc row
with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  


marked_df = pd.DataFrame(marked_data)

values = marked_df.iloc[5] # row 5 here contains indexes for corneas and rhabdoms/eye
print(values)


The next blocks of code actually convert your mat file into a csv. Make sure you include your mat path, the path you want to save your csv to (here 'raw_labels'), and include the correct index values (i.e. for corneas or rhabdoms). 

In [ ]:
# To convert mat for Primno_abyssalis_FEG221018_357 into corneas raw labels csv
import h5py
import pandas as pd
import os

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_357/Primno_abyssalis_357.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  

marked_df = pd.DataFrame(marked_data)


columns_to_keep = marked_df.iloc[5].isin([0, 2, 4, 6]) # correct indices for corneas
column_indices = marked_df.columns[columns_to_keep]
filtered_df = marked_df.iloc[:3, column_indices]
transposed_df = filtered_df.transpose()

# Specify directory path
save_directory = '/home/jake/projects/dhr/dataset/raw_labels'  
csv_name = 'Primno_abyssalis_FEG221018_357_labels_corneas.csv' # name csv 


csv_path = os.path.join(save_directory, csv_name)


os.makedirs(save_directory, exist_ok=True)


transposed_df.to_csv(csv_path, header=False, index=False)


In [ ]:
# To convert mat for Primno_abyssalis_FEG221018_356 into corneas raw labels csv
import h5py
import pandas as pd
import os

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

marked_df = pd.DataFrame(marked_data)


columns_to_keep = marked_df.iloc[5].isin([0, 2, 4, 6]) # correct indices for corneas
column_indices = marked_df.columns[columns_to_keep]
filtered_df = marked_df.iloc[:3, column_indices]
transposed_df = filtered_df.transpose()

# Specify the directory where you want to save the CSV
save_directory = '/home/jake/projects/dhr/dataset/raw_labels'  
csv_name = 'Primno_abyssalis_FEG221018_356_labels_corneas.csv'

csv_path = os.path.join(save_directory, csv_name)

os.makedirs(save_directory, exist_ok=True)

transposed_df.to_csv(csv_path, header=False, index=False)


The next block of code generate your dataset 

In [ ]:
# !python generate_dataset.py -h #(dont forget you have generated the corneas datset using rhabdoms csv paths in the corneas data source specifiers csv)
!python generate_dataset.py ./data_source_specifiers/primno_corneas.csv -l corneas -v 10 -cl corneas -cb 16

## Checking

In [ ]:
!python check_data.py ./configs/primno_corneas.yaml

## Training

In [ ]:
'''version 1'''
# !python main.py train configs/primno_corneas.yaml -w ./logs/cystisoma_corneas/lightning_logs/version_13/checkpoints/epoch=2758-step=696256.ckpt -ne 10
'''version 3'''
!python main.py train configs/primno_corneas.yaml -w ./logs/cystisoma_corneas/lightning_logs/version_14/checkpoints/epoch=3589-step=799360-v1.ckpt -ne 10


## Inference

In [ ]:
import pandas as pd

#Read in your csv and assign it to the variable RR
# 2nd scan
RR = pd.read_csv('/home/jake/projects/dhr/Primno_abyssalis_FEG221101_359_corneas_estimating_resampling_ratio.csv')

RR

In [ ]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-0', 'axis-1', 'axis-2']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix between points
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance
resample_ratio = mean_nearest_distance/10
resample_ratio #Print your RR; use this to make your inference 

In [ ]:
# Inferences with version 1 (bad, model not used)
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_1/ -rr 3.290426824776052 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.5
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_1/ -rr 3.290426824776052 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.1


In [ ]:
# Infernces with version 3
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.1 -at 0.05
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.1 -at 0.01
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.1 -at 0.5
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.1 -at 0.4
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.4 -at 0.5 --infer_with_max_filter_fun
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.3 -at 0.5 --infer_with_max_filter_fun
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.2 -at 0.5 --infer_with_max_filter_fun
# !python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 2 -ny 2 -nz 2 -ipmv 0.1 -at 0.5 --infer_with_max_filter_fun # best so far
!python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 3 -ny 3 -nz 3 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
!python main.py infer configs/primno_corneas.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_corneas/lightning_logs/version_3/ -rr 3.290426824776052 -nx 3 -ny 3 -nz 3 -ipmv 0.1 -at 0.5 --infer_with_max_filter_fun


In [ ]:
import napari
import numpy as np
import torchio as tio


image_path = '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii'
image = tio.ScalarImage(image_path)

# heatmap_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction.nii'
heatmap_path = '/home/jake/projects/dhr/output/Primno_abyssalis_FEG221101_359.logs_primno_corneas_lightning_logs_version_3_checkpoints_last_x_2_y_2_z_2_average_threshold_0.5_prediction.nii'

#Make sure to use resampled space csv here.
# y_hat_path = '/media/jake/Removable backup/Jake/projects/dhr/output/FEG20190213_010.logs_cystisoma_rhabdoms_lightning_logs_version_17_checkpoints_last_x_3_y_3_z_3_average_threshold_0.5_prediction_peak_min_val_0_5_method_center_of_mass.resampled_space_peaks.csv'
y_hat_path = '/home/jake/projects/dhr/output/Primno_abyssalis_FEG221101_359.logs_primno_corneas_lightning_logs_version_3_checkpoints_last_x_2_y_2_z_2_average_threshold_0.5_prediction_peak_min_val_0_4_method_max_filter.resampled_space_peaks.csv'

heatmap = tio.ScalarImage(heatmap_path)


viewer = napari.view_image(heatmap.numpy(), name='heatmap', contrast_limits=[0, 1])
# viewer.add_image(image.numpy(), name='image') #If you want to view your scan also (not necessary)

# load the coordinates to analyse
def load_coordinates(path, flip_axes=False, mct_path=None):
    locations = np.loadtxt(
        path,
        delimiter=',',
        ndmin=2,
        dtype=np.float64
    )
    if not flip_axes: 
        return locations.tolist()

    if not mct_path:
        raise Exception('You must specify `mct_path` if you need to flip_axes')

    mct = tio.ScalarImage(mct_path)
    locations[:,0] = mct.shape[1] - locations[:,0]
    locations[:,1] = mct.shape[2] - locations[:,1]

    return locations.tolist()


y_hat = load_coordinates(y_hat_path)


viewer.add_points(y_hat)

# Primno rhabdoms

## Generating the dataset

The next couple kernels are to manually convert mat files into csv files with rhabdoms points. If you do not need to convert manually you can use `!python mctv_to_csv.py`.

In [ ]:
# To convert mat for Primno_abyssalis_FEG221018_356 into rhabdoms raw labels csv
import h5py
import pandas as pd
import os

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_356/Primno_abyssalis_356_eye.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  # Load the entire marked dataset

marked_df = pd.DataFrame(marked_data)


columns_to_keep = marked_df.iloc[5].isin([1, 3, 5, 7]) # correct indices for rhabdoms
column_indices = marked_df.columns[columns_to_keep]
filtered_df = marked_df.iloc[:3, column_indices]
transposed_df = filtered_df.transpose()

# Specify the directory where you want to save the CSV
save_directory = '/home/jake/projects/dhr/dataset/raw_labels'  
csv_name = 'Primno_abyssalis_FEG221018_356_labels_rhaddoms.csv'

csv_path = os.path.join(save_directory, csv_name)

os.makedirs(save_directory, exist_ok=True)

transposed_df.to_csv(csv_path, header=False, index=False)


In [ ]:
# To convert mat for Primno_abyssalis_FEG221018_357 into rhabdoms raw labels csv
import h5py
import pandas as pd
import os

with h5py.File('/media/jake/Removable backup/Jake/projects/mctv_resfiles/hyperiids/Primno_abyssalis_FEG221018_357/Primno_abyssalis_357.mat', 'r') as file:
    marked_data = file['save_dat/data/marked'][:]  

marked_df = pd.DataFrame(marked_data)


columns_to_keep = marked_df.iloc[5].isin([1, 3, 5, 7]) # correct indices for rhabdoms
column_indices = marked_df.columns[columns_to_keep]
filtered_df = marked_df.iloc[:3, column_indices]
transposed_df = filtered_df.transpose()

# Specify directory path
save_directory = '/home/jake/projects/dhr/dataset/raw_labels'  
csv_name = 'Primno_abyssalis_FEG221018_357_labels_rhabdoms.csv' # name csv 


csv_path = os.path.join(save_directory, csv_name)


os.makedirs(save_directory, exist_ok=True)


transposed_df.to_csv(csv_path, header=False, index=False)


In [ ]:
!python generate_dataset.py -h

In [ ]:
!python generate_dataset.py ./data_source_specifiers/primno_rhabdoms.csv -l rhabdoms -v 10 -cl rhabdoms -cb 16

## Checking

In [ ]:
!python check_data.py ./configs/primno_rhabdoms.yaml

## Training

In [ ]:
# Version 1
!python main.py train configs/primno_rhabdoms.yaml -w ./logs/cystisoma_rhabdoms/lightning_logs/version_19/checkpoints/epoch=1779-step=1004321.ckpt -ne 10


In [ ]:
# Version 2
!python main.py train configs/primno_rhabdoms.yaml -w ./logs/paraphronima_rhabdoms_from_pretrained/lightning_logs/version_1/checkpoints/epoch=147-step=890880.ckpt -ne 10
# Version 3
# !python main.py train configs/primno_rhabdoms.yaml -w ./logs/paraphronima_rhabdoms/lightning_logs/version_23/checkpoints/epoch=99-step=537600-v1.ckpt -ne 10


## Inference

In [ ]:
import torchio as tio

image_path = '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii'
image = tio.ScalarImage(image_path)

# view file with napari
import napari


viewer = napari.view_image(image.numpy(), name='image')

In [ ]:
import pandas as pd

RR = pd.read_csv('/home/jake/projects/dhr/Primno_abyssalis_FEG221101_359_corneas_estimating_resampling_ratio.csv')

RR

In [ ]:
from scipy.spatial import distance_matrix

coordinates = RR[['axis-0', 'axis-1', 'axis-2']].values
dist_matrix = distance_matrix(coordinates, coordinates) #Create a distance matrix between points
import numpy as np
np.fill_diagonal(dist_matrix, np.inf)

# Find the nearest neighbor distance for each point
nearest_distances = np.min(dist_matrix, axis=1)

# Add the nearest distances to the dataframe
RR['nearest_distance'] = nearest_distances
RR
mean_nearest_distance = RR['nearest_distance'].mean()
mean_nearest_distance
resample_ratio = mean_nearest_distance/10
resample_ratio #Print your RR; use this to make your inference 

In [ ]:
!python main.py infer configs/primno_rhabdoms.yaml -v '/media/jake/Removable backup/Dropbox/Full body hyperiid scans/Head Scans_nifti/Primno_abyssalis_f359_u0000000/Primno_abyssalis_FEG221101_359.nii' -m ./logs/primno_rhabdoms/lightning_logs/version_1/ -rr 2.9493045478545903 -nx 1 -ny 1 -nz 1 -ipmv 0.5 -at 0.5 --infer_with_max_filter_fun
